## Setup and configuration

### Critical Fixes Applied:

1. **Pre-sample Training Data**: Sample training data in main process before sending to workers - workers receive only ~2.5-27.5% of full pool (much faster serialization)
2. **DataFrame → Numpy Conversion**: Convert to numpy arrays before multiprocessing to avoid pickle overhead
3. **Founder Simplification**: Founder model establishes baseline score but is NOT added to ensemble, simplifying batch/iteration indexing
4. **Reduced Batch Size**: Lowered from 10 to 5 workers to reduce memory pressure
5. **Enhanced Error Handling**: Better progress tracking and graceful handling of failed models
6. **Empty Ensemble Start**: Ensemble starts empty at iteration 1, avoiding index confusion

### Serialization Impact:
- **Before**: 5 workers × 40,000 rows × 30 cols = ~48 MB per batch
- **After**: 5 workers × ~6,000 rows × 30 cols = ~7 MB per batch (~85% reduction!)


In [1]:
# Standard library imports
import os
import sys
import time
from datetime import datetime
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import cpu_count

# Disable GPU and limit threading
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['OMP_NUM_THREADS'] = '2'

# Third party imports
import joblib
import json
import pandas as pd
import tensorflow as tf

# Import ensemble modules
from functions import ensemble_database
from functions.ensemble_initialization import create_data_splits, create_base_preprocessor, train_founder_model
from functions.ensemble_parallel import train_single_candidate, prepare_training_batch
from functions.ensemble_evaluation import evaluate_candidate_ensemble
from functions.ensemble_stage2_training import train_or_expand_stage2_model, save_ensemble_bundle
from functions.ensemble_hill_climbing import (
    adaptive_simulated_annealing_acceptance,
    update_temperature,
    log_iteration
)
from functions.ensemble_stage2_model import save_checkpoint

# Configure TensorFlow
tf.get_logger().setLevel('ERROR')

# Detect available CPUs
n_cpus = cpu_count()
print(f"TensorFlow version: {tf.__version__}")
print(f"Available CPUs: {n_cpus}")
print(f"GPU disabled: CUDA drivers not available in dev container")

2025-12-07 07:38:27.268638: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765093107.290303   71271 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765093107.297078   71271 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow version: 2.18.0
Available CPUs: 24
GPU disabled: CUDA drivers not available in dev container


### Configuration parameters

In [ ]:
# Random state for reproducibility
RANDOM_STATE = 315

# CPU allocation for parallel training
# Set to None to use all available CPUs, or specify a number to limit
N_CPUS = 10  # Will use all available cores by default

if N_CPUS is None:
    import multiprocessing
    N_CPUS = multiprocessing.cpu_count()

# Parallel training configuration
BATCH_SIZE = 5  # Train this many candidates in parallel (reduced from 10 to avoid serialization issues)
N_WORKERS = min(BATCH_SIZE, N_CPUS)

# Hill climbing configuration
MAX_ITERATIONS = 500
PLATEAU_ITERATIONS = 100
BASE_TEMPERATURE = 0.01
TEMPERATURE_DECAY = 0.995

# Stage 2 DNN configuration
STAGE2_BATCH_SIZE_MODELS = 10  # Retrain DNN every N accepted models
STAGE2_EPOCHS = 100
STAGE2_BATCH_SIZE = 128
STAGE2_PATIENCE = 10

# Paths
DATA_DIR = Path('../data')
MODELS_BASE_DIR = Path('../models')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
MODELS_DIR = MODELS_BASE_DIR / f'run_{timestamp}'
ENSEMBLE_DIR = MODELS_DIR / 'ensemble_stage1_models'
CHECKPOINT_PATH = MODELS_DIR / 'ensemble_checkpoint.pkl'

# Create directories
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)
ENSEMBLE_DIR.mkdir(parents=True, exist_ok=True)

# Initialize database
ensemble_database.reset_database()
ensemble_database.init_database()

print(f"\nConfiguration:")
print(f"  Total CPUs available: {N_CPUS}")
print(f"  Parallel workers: {N_WORKERS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Note: CPU cores will be distributed intelligently across models in each batch")
print(f"        - RandomForest, KNN, ExtraTrees get more cores (faster training)")
print(f"        - Logistic, SVM, etc. get 1 core (no parallelization benefit)")

Database initialized at: /workspaces/diabetes-prediction/data/ensemble_training.db
Database initialized at: /workspaces/diabetes-prediction/data/ensemble_training.db

Configuration:
  Random state: 315
  Parallel: 10 candidates, 10 workers
  Max iterations: 500
  Training run directory: ../models/run_20251207_073830


### Diagnostic: Check data types before parallel processing

In [ ]:
print("Data info before batch preparation:")
print(f"  X_train_pool: {type(X_train_pool)} - {X_train_pool.shape if hasattr(X_train_pool, 'shape') else 'N/A'}")
print(f"  y_train_pool: {type(y_train_pool)} - {y_train_pool.shape if hasattr(y_train_pool, 'shape') else 'N/A'}")
print(f"  X_val_s1: {type(X_val_s1)} - {X_val_s1.shape if hasattr(X_val_s1, 'shape') else 'N/A'}")
print(f"\nOptimizations:")
print(f"  1. prepare_training_batch will convert DataFrames → numpy arrays")
print(f"  2. Each worker receives PRE-SAMPLED data (~2.5-27.5% of pool)")
print(f"  3. Estimated data per worker: ~{len(X_train_pool) * 0.15:.0f} rows (vs {len(X_train_pool)} full pool)")


## Data loading and preparation

In [3]:
# Load training data
train_df_path = 'https://gperdrizet.github.io/FSA_devops/assets/data/unit3/diabetes_prediction_train.csv'
train_df = pd.read_csv(train_df_path)
train_df.drop_duplicates(inplace=True)

print(f"Training data shape: {train_df.shape}")
print(f"Class distribution:")
print(train_df['diagnosed_diabetes'].value_counts(normalize=True))

# Define features
label = 'diagnosed_diabetes'
numerical_features = [
    'age', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week',
    'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi',
    'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
    'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides',
    'family_history_diabetes', 'hypertension_history', 'cardiovascular_history'
]
ordinal_features = ['education_level', 'income_level']
education_categories = [['No formal', 'Highschool', 'Graduate', 'Postgraduate']]
income_categories = [['Low', 'Lower-Middle', 'Middle', 'Upper-Middle', 'High']]
nominal_features = ['gender', 'ethnicity', 'smoking_status', 'employment_status']

Training data shape: (700000, 26)
Class distribution:
diagnosed_diabetes
1.0    0.623296
0.0    0.376704
Name: proportion, dtype: float64


In [4]:
# Create fixed three-way data split
X_train_pool, X_val_s1, X_val_s2, y_train_pool, y_val_s1, y_val_s2 = create_data_splits(
    train_df, label, RANDOM_STATE
)


Fixed data split:
--------------------------------------------------------------------------------
  Training pool: 420,000 samples (60%)
  Stage 1 validation: 140,000 samples (20%) - for stage 1 eval & stage 2 training
  Stage 2 validation: 140,000 samples (20%) - for stage 2 eval (HELD OUT)


In [5]:
# Create base preprocessor
base_preprocessor = create_base_preprocessor(
    numerical_features, ordinal_features, nominal_features,
    education_categories, income_categories
)


Base preprocessor created
  Numerical features: 18
  Ordinal features: 2
  Nominal features: 4


## Initialize ensemble with founder model

In [ ]:
# Train founder model (baseline only - NOT added to ensemble)
founder_auc = train_founder_model(
    X_train_pool, X_val_s1, X_val_s2, y_train_pool, y_val_s1, y_val_s2,
    base_preprocessor, RANDOM_STATE, BASE_TEMPERATURE, ENSEMBLE_DIR
)

# Initialize ensemble (EMPTY - founder not included)
ensemble_models = []
stage2_model = None
best_ensemble_score = founder_auc

# Initialize hill climbing variables (start at iteration 1, not 0)
start_iteration = 1
temperature = BASE_TEMPERATURE

print(f"\nFounder baseline AUC: {founder_auc:.6f}")
print(f"Ensemble starts empty - first batch will be iterations 1-{BATCH_SIZE}")
print(f"Stage 2 DNN will be trained after {STAGE2_BATCH_SIZE_MODELS} accepted models")


INITIALIZING FOUNDER MODEL

Training founder model
--------------------------------------------------------------------------------
  Training samples: 39,283
  Pipeline config:
    Classifier: knn
    Transformers: ['sqrt', 'square']
    Dimensionality reduction: factor_analysis
  Training pipeline...

Training founder model
--------------------------------------------------------------------------------
  Training samples: 39,283
  Pipeline config:
    Classifier: knn
    Transformers: ['sqrt', 'square']
    Dimensionality reduction: factor_analysis
  Training pipeline...
  Training complete (8.4s)
  Training complete (8.4s)
  Stage 1 validation AUC: 0.515608
  Stage 2 validation AUC: 0.516098

FOUNDER MODEL COMPLETE

Founder model AUC: 0.516098
Stage 2 DNN will be trained after 10 accepted models
  Stage 1 validation AUC: 0.515608
  Stage 2 validation AUC: 0.516098

FOUNDER MODEL COMPLETE

Founder model AUC: 0.516098
Stage 2 DNN will be trained after 10 accepted models


## Parallel hill climbing loop

Iteratively trains batches of candidate models in parallel, evaluates with hybrid scoring,
and accepts/rejects using simulated annealing.

In [ ]:
print(f"\n{'=' * 80}")
print("STARTING PARALLEL HILL CLIMBING LOOP")
print(f"{'=' * 80}")
print(f"Batch size: {BATCH_SIZE} candidates trained in parallel")
print(f"Workers: {N_WORKERS} parallel processes")
print(f"Total CPUs: {N_CPUS} (distributed intelligently across models)")

iterations_since_improvement = 0
iteration = start_iteration

while iteration < MAX_ITERATIONS and iterations_since_improvement < PLATEAU_ITERATIONS:
    print(f"\n{'=' * 80}")
    print(f"BATCH Starting at iteration {iteration}")
    print(f"{'=' * 80}")
    print(f"Ensemble size: {len(ensemble_models)} | Best score: {best_ensemble_score:.6f} | "
          f"Temperature: {temperature:.6f} | No improvement: {iterations_since_improvement}/{PLATEAU_ITERATIONS}")
    
    # Prepare batch of training jobs with intelligent CPU allocation
    print(f"\nPreparing batch jobs...")
    batch_jobs = prepare_training_batch(
        iteration, BATCH_SIZE, MAX_ITERATIONS, X_train_pool, y_train_pool,
        X_val_s1, y_val_s1, base_preprocessor, RANDOM_STATE, total_cpus=N_CPUS
    )
    print(f"Batch prepared: {len(batch_jobs)} jobs ready")
    
    print(f"\nTraining {len(batch_jobs)} candidates in parallel...")
    batch_start_time = time.time()
    
    # Train candidates in parallel
    trained_candidates = []
    failed_count = 0
    with ProcessPoolExecutor(max_workers=N_WORKERS) as executor:
        futures = {executor.submit(train_single_candidate, job): job for job in batch_jobs}
        
        completed = 0
        for future in as_completed(futures):
            completed += 1
            try:
                result = future.result()
                trained_candidates.append(result)
                print(f"  [{completed}/{len(batch_jobs)}] ✓ Iteration {result['iteration']}: "
                      f"{result['metadata']['classifier_type']} AUC={result['val_auc_s1']:.6f} "
                      f"({result['training_time']:.1f}s)")
            except Exception as e:
                failed_count += 1
                job = futures[future]
                print(f"  [{completed}/{len(batch_jobs)}] ✗ Iteration {job[0]} FAILED: {e}")
    
    if failed_count > 0:
        print(f"\n⚠️  {failed_count}/{len(batch_jobs)} models failed during training")
    
    if len(trained_candidates) == 0:
        print(f"\n❌ CRITICAL: All models in batch failed! Stopping.")
        break
    
    batch_time = time.time() - batch_start_time
    print(f"\nBatch complete ({batch_time:.1f}s, {batch_time/len(trained_candidates):.1f}s per model)")
    
    # Sort by iteration number
    trained_candidates.sort(key=lambda x: x['iteration'])
    
    # Process each trained candidate for acceptance/rejection
    for result in trained_candidates:
        current_iter = result['iteration']
        fitted_pipeline = result['fitted_pipeline']
        metadata = result['metadata']
        val_auc_s1 = result['val_auc_s1']
        pipeline_hash = result['pipeline_hash']
        training_memory_mb = result.get('memory_mb', None)
        training_time_sec = result.get('training_time_sec', None)
        
        print(f"\n{'-' * 80}")
        print(f"Iteration {current_iter}: {metadata['classifier_type']} | Stage 1 AUC: {val_auc_s1:.6f}")
        
        # Evaluate ensemble with candidate
        if len(ensemble_models) == 0:
            # First model - just use its score
            candidate_score = val_auc_s1
            diversity_score = 0.0
            aggregation_method = "single_model"
            print(f"  Ensemble AUC ({aggregation_method}): {candidate_score:.6f} (first model)")
        else:
            # Evaluate as ensemble
            candidate_ensemble = ensemble_models + [fitted_pipeline]
            candidate_score, diversity_score, aggregation_method = evaluate_candidate_ensemble(
                candidate_ensemble, ensemble_models, stage2_model,
                X_val_s1, X_val_s2, y_val_s1, y_val_s2
            )
            print(f"  Ensemble AUC ({aggregation_method}): {candidate_score:.6f} | Diversity: {diversity_score:.6f}")
        
        # Simulated annealing acceptance
        accept, reason = adaptive_simulated_annealing_acceptance(
            current_score=best_ensemble_score,
            candidate_score=candidate_score,
            temperature=temperature,
            random_state=RANDOM_STATE + current_iter
        )
        
        print(f"  Decision: {'✓ ACCEPT' if accept else '✗ REJECT'} ({reason})")
        
        # Track stage 2 memory and time for this iteration
        stage2_memory_mb = None
        stage2_time_sec = None
        
        # Log iteration
        log_iteration(
            iteration=current_iter,
            accepted=accept,
            rejection_reason=reason,
            pipeline_hash=pipeline_hash,
            stage1_val_auc=val_auc_s1,
            stage2_val_auc=candidate_score,
            ensemble_size=len(ensemble_models) + 1 if accept else len(ensemble_models),
            diversity_score=diversity_score,
            temperature=temperature,
            metadata=metadata,
            ensemble_id=f"iter_{current_iter}",
            training_memory_mb=training_memory_mb,
            stage2_memory_mb=stage2_memory_mb,
            training_time_sec=training_time_sec,
            stage2_time_sec=stage2_time_sec
        )
        
        # Update ensemble if accepted
        if accept:
            ensemble_models.append(fitted_pipeline)
            
            # Save model
            model_path = ENSEMBLE_DIR / f'model_{current_iter}.joblib'
            joblib.dump(fitted_pipeline, model_path)
            
            # Check if we should train/retrain stage 2 DNN
            if len(ensemble_models) % STAGE2_BATCH_SIZE_MODELS == 0 and len(ensemble_models) > 0:
                stage2_model, final_score, stage2_memory_mb, stage2_time_sec = train_or_expand_stage2_model(
                    ensemble_models, stage2_model, X_val_s1, y_val_s1, X_val_s2, y_val_s2,
                    STAGE2_EPOCHS, STAGE2_BATCH_SIZE, STAGE2_PATIENCE, current_iter
                )
                
                # Save ensemble bundle checkpoint
                save_ensemble_bundle(
                    ensemble_models, stage2_model, best_ensemble_score, current_iter,
                    MODELS_DIR, RANDOM_STATE, BATCH_SIZE, N_WORKERS, base_preprocessor,
                    numerical_features, ordinal_features, nominal_features,
                    education_categories, income_categories
                )
                print(f"{'=' * 80}\n")
            
            # Check if this is the best score
            if candidate_score > best_ensemble_score:
                print(f"  🎉 New best score: {candidate_score:.6f} (Δ={candidate_score - best_ensemble_score:.6f})")
                best_ensemble_score = candidate_score
                iterations_since_improvement = 0
            else:
                iterations_since_improvement += 1
        else:
            iterations_since_improvement += 1
        
        # Update temperature
        temperature = update_temperature(
            iteration=current_iter,
            acceptance_history=[accept],
            current_temperature=temperature,
            base_temperature=BASE_TEMPERATURE,
            decay_rate=TEMPERATURE_DECAY
        )
    
    # Move to next batch
    iteration += len(trained_candidates)
    
    # Check termination
    if iterations_since_improvement >= PLATEAU_ITERATIONS:
        print(f"\n{'=' * 80}")
        print(f"TERMINATING: No improvement for {PLATEAU_ITERATIONS} iterations")
        print(f"{'=' * 80}")
        break

# Calculate final acceptance rate
conn = ensemble_database.sqlite3.connect(ensemble_database.DB_PATH)
acceptance_stats = conn.execute("SELECT COUNT(*) as total, SUM(accepted) as accepted FROM ensemble_log WHERE iteration_num > 0").fetchone()
conn.close()
acceptance_rate = acceptance_stats[1] / acceptance_stats[0] if acceptance_stats[0] > 0 else 0.0

print(f"\n{'=' * 80}")
print("HILL CLIMBING COMPLETE")
print(f"{'=' * 80}")
print(f"Final ensemble size: {len(ensemble_models)}")
print(f"Best ensemble AUC: {best_ensemble_score:.6f}")
print(f"Total iterations: {iteration - 1}")
print(f"Acceptance rate: {acceptance_rate:.1%}")


STARTING PARALLEL HILL CLIMBING LOOP
Batch size: 10 candidates trained in parallel
Workers: 10 parallel processes

BATCH Starting at iteration 1
Ensemble size: 1 | Best score: 0.516098 | Temperature: 0.010000 | No improvement: 0/100

Training 10 candidates in parallel...
  ✓ Iteration 1: sgd_classifier AUC=0.587573 (2.6s)
  ✓ Iteration 1: sgd_classifier AUC=0.587573 (2.6s)
  ✓ Iteration 7: mlp AUC=0.538933 (7.9s)
  ✓ Iteration 10: lda AUC=0.504031 (6.4s)
  ✓ Iteration 7: mlp AUC=0.538933 (7.9s)
  ✓ Iteration 10: lda AUC=0.504031 (6.4s)
  ✓ Iteration 4: sgd_classifier AUC=0.500000 (15.8s)
  ✓ Iteration 4: sgd_classifier AUC=0.500000 (15.8s)
  ✓ Iteration 3: random_forest AUC=0.604389 (129.1s)
  ✓ Iteration 3: random_forest AUC=0.604389 (129.1s)
  ✗ Iteration 8 failed: Training exceeded 5 minute timeout (classifier: logistic_regression)
  ✗ Iteration 8 failed: Training exceeded 5 minute timeout (classifier: logistic_regression)
  ✗ Iteration 2 failed: Training exceeded 5 minute timeout 

## Save final checkpoint and bundle

In [ ]:
# Save final checkpoint
save_checkpoint(
    checkpoint_path=CHECKPOINT_PATH,
    ensemble_models=ensemble_models,
    stage2_model=stage2_model,
    iteration=iteration - 1,
    temperature=temperature,
    best_score=best_ensemble_score,
    acceptance_history=[],
    metadata={
        'total_iterations': iteration,
        'final_ensemble_size': len(ensemble_models),
        'acceptance_rate': acceptance_rate,
        'best_score': best_ensemble_score,
        'parallel_batch_size': BATCH_SIZE,
        'n_workers': N_WORKERS
    }
)

# Save metadata
metadata_path = MODELS_DIR / 'ensemble_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump({
        'ensemble_size': len(ensemble_models),
        'total_iterations': iteration,
        'best_score': best_ensemble_score,
        'acceptance_rate': acceptance_rate,
        'training_completed': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'random_state': RANDOM_STATE,
        'parallel_batch_size': BATCH_SIZE,
        'n_workers': N_WORKERS
    }, f, indent=2)

print(f"\nCheckpoint saved: {CHECKPOINT_PATH}")
print(f"Metadata saved: {metadata_path}")

In [ ]:
# Save final ensemble bundle for Kaggle
ensemble_bundle_path = MODELS_DIR / 'ensemble_bundle.joblib'

ensemble_bundle = {
    'ensemble_models': ensemble_models,
    'stage2_model': stage2_model,
    'metadata': {
        'ensemble_size': len(ensemble_models),
        'total_iterations': iteration,
        'best_score': best_ensemble_score,
        'acceptance_rate': acceptance_rate,
        'training_completed': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'random_state': RANDOM_STATE,
        'parallel_batch_size': BATCH_SIZE,
        'n_workers': N_WORKERS
    },
    'base_preprocessor': base_preprocessor,
    'feature_info': {
        'numerical_features': numerical_features,
        'ordinal_features': ordinal_features,
        'nominal_features': nominal_features,
        'education_categories': education_categories,
        'income_categories': income_categories
    }
}

joblib.dump(ensemble_bundle, ensemble_bundle_path, compress=3)

print(f"\nFinal ensemble bundle saved: {ensemble_bundle_path}")
print(f"File size: {ensemble_bundle_path.stat().st_size / (1024**2):.1f} MB")
print(f"\nTo load on Kaggle:")
print(f"  ensemble_bundle = joblib.load('ensemble_bundle.joblib')")
print(f"  ensemble_models = ensemble_bundle['ensemble_models']")
print(f"  stage2_model = ensemble_bundle['stage2_model']")

## Summary

In [ ]:
print(f"\n{'=' * 80}")
print("ENSEMBLE TRAINING SUMMARY")
print(f"{'=' * 80}")
print(f"\nFinal Statistics:")
print(f"  Ensemble size: {len(ensemble_models)}")
print(f"  Best validation AUC: {best_ensemble_score:.6f}")
print(f"  Total iterations: {iteration}")
print(f"  Acceptance rate: {acceptance_rate:.1%}")
print(f"  Parallel configuration: {BATCH_SIZE} candidates, {N_WORKERS} workers")
print(f"\nFiles created:")
print(f"  Database: {ensemble_database.DB_PATH}")
print(f"  Models: {ENSEMBLE_DIR}")
print(f"  Checkpoint: {CHECKPOINT_PATH}")
print(f"  Metadata: {metadata_path}")
print(f"  Bundle: {ensemble_bundle_path}")
print(f"\n{'=' * 80}")